In [1]:
cd ..

/home/cms-jovyan/b_lepton_met/analysis


In [14]:
import json
import pickle
import importlib
import numpy as np
import awkward as ak
from typing import List
from coffea import processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.analysis_tools import Weights, PackedSelection
from corrections import add_pileup_weight

In [18]:
class BackgroundEstimatorProcessor(processor.ProcessorABC):
    def __init__(
        self, 
        year: str = "2017", 
        yearmod: str = "",
        channels: List[str] = ["ele", "mu"],
        output_location : str = "./outfiles/"
    ):
        self._year = year
        self._yearmod = yearmod
        self._channels = channels
        self._output_location = output_location
        self.common_weights = ["genweight", "L1Prefiring", "pileup"]
        
        # open triggers
        with open("data/triggers.json", "r") as f:
            self._triggers = json.load(f)[self._year]
            
        # open btagDeepFlavB
        with open("data/btagDeepFlavB.json", "r") as f:
            self._btagDeepFlavB = json.load(f)[self._year]
            
        # open lumi masks
        with open('data/lumi_masks.pkl', 'rb') as handle:
            self._lumi_mask = pickle.load(handle)
        
        
    def add_selection(
        self, 
        name: str, 
        sel: np.ndarray, 
        channel: List[str] = None
    ):
        """Adds selection to PackedSelection object and the cutflow dictionary"""
        channels = channel if channel else self._channels
        for ch in channels:
            self.selections[ch].add(name, sel)
            selection_ch = self.selections[ch].all(*self.selections[ch].names)
            if self.isMC:
                weight = self.weights.partial_weight(self.weights_per_ch[ch] + self.common_weights)
                self.cutflows[ch][name] = float(weight[selection_ch].sum())
            else:
                self.cutflows[ch][name] = np.sum(selection_ch)
        
        
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, events: ak.Array):
        """Returns skimmed events which pass preselection cuts and with the branches listed in self._skimvars"""
        dataset = events.metadata["dataset"]
        nevents = len(events)
        
        self.isMC = hasattr(events, "genWeight")
        self.weights = Weights(nevents, storeIndividual=True)
        self.weights_per_ch = {}
        self.selections = {}
        self.cutflows = {}
        for ch in self._channels:
            self.weights_per_ch[ch] = []
            self.selections[ch] = PackedSelection()
            self.cutflows[ch] = {}

        sumgenweight = ak.sum(events.genWeight) if self.isMC else 0
        
        # lumi 
        if not self.isMC:
            self._lumi = self._lumi_mask[self._year](events.run, events.luminosityBlock)
        else:
            self._lumi = np.ones(len(events), dtype='bool')

        # triggers
        trigger = {}
        for ch in self._channels:
            trigger[ch] = np.zeros(nevents, dtype="bool")
            for t in self._triggers[ch]:
                if t in events.HLT.fields:
                    trigger[ch] = trigger[ch] | events.HLT[t]
            
            
        # electrons 
        good_electrons = (
            (events.Electron.pt > 0)
            & (np.abs(events.Electron.eta) < 2.4)
            & ((np.abs(events.Electron.eta) < 1.44) | (np.abs(events.Electron.eta) > 1.57))
            & (events.Electron.mvaFall17V2noIso_WP90)
        )
        n_good_electrons = ak.sum(good_electrons, axis=1)
        
        
        # muons
        # mediumId OR tightId?
        # DO WE NEED LOOSE MUONS? 
        good_muons = (
            (events.Muon.pt > 30)
            & (np.abs(events.Muon.eta) < 2.4)
            & events.Muon.mediumId
        )
        n_good_muons = ak.sum(good_muons, axis=1)
        
        # get candidate lepton
        goodleptons = ak.concatenate(
            [events.Muon[good_muons], events.Electron[good_electrons]], axis=1
        )  
        goodleptons = goodleptons[ak.argsort(goodleptons.pt, ascending=False)]  
        candidatelep = ak.firsts(goodleptons)
        
        
        # b-jets
        # IS btagDeepFlavB YEAR AND CHANNEL DEPENDENT?
        good_bjets = (
            (ak.firsts(events.Jet.pt) > 30)
            & (events.Jet.jetId == 6)
            & (events.Jet.puId == 7)
            & (events.Jet.btagDeepFlavB > self._btagDeepFlavB)
        )
        n_good_bjets = ak.sum(good_bjets, axis=1)
        
        # MET
        met = events.MET[events.MET.pt > 50]
        
        
        # weights
        weigths = {}
        if self.isMC:
            self.weights.add("genweight", events.genWeight)
            if self._year in ("2016", "2017"):
                self.weights.add(
                    "L1Prefiring", events.L1PreFiringWeight.Nom, events.L1PreFiringWeight.Up, events.L1PreFiringWeight.Dn
                )
            add_pileup_weight(self.weights, self._year, self._yearmod, nPU=ak.to_numpy(events.Pileup.nPU))
            
                    
        # selections
        self.add_selection("all", np.ones(nevents, dtype="bool"))
        for ch in self._channels:
            self.add_selection("trigger", trigger[ch], [ch])
        self.add_selection(
            name="one_lepton",
            sel=(
                (n_good_muons == 1)
                & (n_good_electrons == 0)
            ),
            channel=["mu"],
        )
        self.add_selection(
            name="one_lepton",
            sel=(
                (n_good_muons == 0)
                & (n_good_electrons == 1)
            ),
            channel=["ele"],
        )
        self.add_selection("two_bjets", n_good_bjets >= 2)
        
        
        ## output
        #output = {}
        #for ch in self._channels:
        #    selection_ch = self.selections[ch].all(*self.selections[ch].names)
        #    output[ch] = {"jet": }
            
        return {dataset: weigths}
            
    def postprocess(self, accumulator):
        return accumulator

In [19]:
fileset = {
    "DYJetsToLL_Pt-100To250_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8": ["root://xcache//store/mc/RunIISummer20UL17NanoAODv2/DYJetsToLL_Pt-100To250_MatchEWPDG20_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v8-v1/240000/A3975393-1AED-B547-8CA2-28B5841C4895.root"],
}

In [20]:
out = processor.run_uproot_job(
    fileset,
    treename="Events",
    processor_instance=BackgroundEstimatorProcessor(),
    executor=processor.IterativeExecutor,
    executor_args={
        "schema": processor.NanoAODSchema,
    },
)

Output()

[False False False ... False False False]

[False False False ... False False False]

[False False False ... False False False]

[False False False ... False False False]

[False False False ... False False False]

[False False False ... False False False]